#  **SM64 Speedruns - A SQL and Python Project**

\** **Ignore any redundant / unnecessary comments, they're mainly just notes for me because I am a noob :]**

Learning Python for data analysis / science and wanted to test what I've learned so far on a personal project using a dataset from Kaggle. (https://www.kaggle.com/code/mcpenguin/super-mario-64-speedruns-data-collection) This dataset
\
\
Updated the dataset files to include a RUN_ID column to later replace the original "id" column. Updated the PLACE column for each PLATFORM. On speedrun.com the platforms are in separate tabs with their own placing.
\
\
\
I also haven't used git in a while so this is also sort of a guinea pig for re-learning that.

## **Gather and Cleanse Data**

### **SQLite Connection**

In [1]:
import pandas as pd
# Learned the purpose of importing an aliased [module].[interface] and then just the [module], but with a diff alias.
# -- Mainly to access separate operations of the module. i.e. changing the graph's style with mpl, and access the plotting operations with plt.
import matplotlib as mpl
import matplotlib.pyplot as plt

import sqlite3
import os
import dateutil, datetime
import seaborn as sns
import numpy as np

# Connection Object to establish connection to a sqlite3 database.
connObj = sqlite3.connect('SPEEDRUNS.db')
cursorObj = connObj.cursor()

%load_ext sql
%sql sqlite:///SPEEDRUNS.db

In [2]:
# Assigning the datasets location for SM64 Speedruns to a variable called "repo"
repo = r'./Data/'
full_path_to_dataset = os.path.join(repo, 'ALL_CATEGORIES.csv')

# Checking if there is data already in the table (mainly for testing + it's just a sqlite table)
result = %sql SELECT COUNT(*) FROM ALL_CAT_SPEEDRUNS

# Extract the count from the result set
count_result = result[0][0] if result is not None and len(result) > 0 else 0

cursorObj.execute('''CREATE TABLE IF NOT EXISTS ALL_CAT_SPEEDRUNS (
    'run_id' INT16 PRIMARY KEY,
    'Category' CATEGORY,
    'id' INT16,
    'place' INT16,
    'speedrun_link' VARCHAR(200),
    'submitted_date' DATETIME,
    'primary_time_seconds' FLOAT64,
    'real_time_seconds' FLOAT64,
    'player_id' TEXT,
    'player_name' TEXT,
    'player_country' TEXT,
    'platform' CATEGORY,
    'verified' BOOL
    )''')
connObj.commit()

# If there is data, delete it
if count_result > 0:
    if os.path.isfile(full_path_to_dataset):
       os.remove(full_path_to_dataset)
       %sql DELETE FROM ALL_CAT_SPEEDRUNS

 * sqlite:///SPEEDRUNS.db
Done.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: './Data/ALL_CATEGORIES.csv'

### **Merging Separate .CSV Files Into A Single Pandas DataFrame.**

**Gathering Datasets (.CSV)**

In [ ]:
# Lists files within the specified directory, in this case "repo".
files_in_repo = os.listdir(repo)

# Looping through files_in_repo and assignging it to the csv_files List only if the file ends with .csv.
csv_files = [f for f in files_in_repo if f.endswith('.csv')]

# List to hold the list of dataframes / csv files.
df_list = []



---



**Appending Separate .CSV DataFrames to the "df_list" List via For Loop**

Found this online because I wasn't sure of the syntax on doing the loop, and the error handling is good, too.

It all makes sense though - here's my walkthrough:

1.   Looping through the list of `csv_files` within `repo`, assigning each to "`csv`".

2.   The path to the file is created by joining the `repo` path and the .csv filename.

1.   Creating a DataFrame (for each iteration of `csv_files`) using the `read_csv()` function and the `file_path` variable.
2.   The DataFrame is appended to the DataFrame List `df_list`.

1.   `try` / `except` = error handling on the encoding types for the files.

In [ ]:
for csv in csv_files:
    file_path = os.path.join(repo, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            # Learned that "f" before a string allows the use of variables (wrapped in curly braces)
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")



---



**Concatenating the DataFrames and Saving to a Single .CSV File**

In [ ]:
# Concat all data into a single DataFrame
complete_df = pd.concat(df_list, ignore_index=True)

---

### **Cleansing / Restructuring Data**

In [ ]:
# Save the final result to a new .csv file (appears in the G Drive folder after 15-30 sec).
complete_df.to_csv(full_path_to_dataset, index=False)

# Reading in the '/content/drive/MyDrive/Kaggle/Datasets/SM64 Speedruns/ALL_CATEGORIES.csv' file and storing into a dataframe.
df = pd.read_csv(full_path_to_dataset)

In [ ]:
# Dropping columns I do not need. The if statement could check if each col is in the df but i didn't want to list them all. This is just so it doesn't error when testing anyway.
if 'speedrun_link' in df:
    cols_to_drop = ['id',
                    'player_id',
                    'speedrun_link',
                    'primary_time_seconds']
else:
    cols_to_drop = []

df.drop(cols_to_drop, inplace=True, axis=1)

# Renaming the columns a bit.
df = df.rename(columns={'run_id': 'ID', 'Category': 'CATEGORY', 'player_name': 'PLAYER_NAME', 'player_country': 'COUNTRY', 'real_time_seconds': 'RUN_TIME', 'submitted_date': 'SUBMISSION_DATE', 'place': 'PLACE', 'platform': 'PLATFORM', 'verified': 'VERIFIED'})

# Some players don't have their country set up on speedrun.com so sqlite sets these to NaN. I'd rather it be null / none.
cursorObj.execute('''UPDATE ALL_CAT_SPEEDRUNS SET COUNTRY = "" WHERE COUNTRY = "NaN"''')
connObj.commit()

In [ ]:
df.loc[:,'RUN_TIME']

In [ ]:
# Created a function to convert the values in the columns. Before I was looping through the column and executing an UPDATE statement for each row (i'm not as noob now)
def seconds_to_time(sec):
    return str(datetime.timedelta(seconds=sec))

def TrimLastThree(value):
    if '.' in value:
        return value[:-3]
    else:
        return value

# Fixing up datatypes.
# df['PLACE'] = df['PLACE'].astype('int16')
df['ID'] = df['ID'].astype('int16')
df['SUBMISSION_DATE'] = pd.to_datetime(df['SUBMISSION_DATE']).dt.strftime("%Y-%m-%d")
df['CATEGORY'] = df['CATEGORY'].astype('category')
df['PLATFORM'] = df['PLATFORM'].astype('category')
df['VERIFIED'] = df['VERIFIED'].astype('category')
df['COUNTRY'] = df['COUNTRY'].astype('str')
df['PLAYER_NAME'] = df['PLAYER_NAME'].astype('str')
df['RUN_TIME'] = df['RUN_TIME'].apply(seconds_to_time)
df['RUN_TIME'] = df['RUN_TIME'].astype('str')
df['RUN_TIME'] = df['RUN_TIME'].apply(TrimLastThree)

df.loc[:,'RUN_TIME']

**Date / Time Cleanup**

I was originally going to remove the first two chars as well (the 0:) hours for 0/1 star, but then realized that there are times over an hour.

In [ ]:
# Updating the table with our latest updates to the dataframe.
df.to_sql('ALL_CAT_SPEEDRUNS', connObj, if_exists='replace', index=False)

## **Analyze Data (mainly a fun SQL test I made)**

In [ ]:
# Baseline table post-cleanse. This is what is currently in ALL_CAT_SPEEDRUNS.
df.info()

#### **Q1: Filter for only Verified runs, how many non-verified runs are there?**

In [ ]:
%%sql  
/*Only verified runs + the total count of all verified and non-verified runs.*/

/*
    VERIFIED: 2097
    NON-VERIFIED: 177
*/
    
SELECT *,
    (SELECT COUNT(*) FROM ALL_CAT_SPEEDRUNS WHERE VERIFIED = "Yes") AS TOTAL_VERIFIED_COUNT,
    (SELECT COUNT(*) FROM ALL_CAT_SPEEDRUNS WHERE VERIFIED = "No") AS TOTAL_NON_VERIFIED_COUNT
FROM ALL_CAT_SPEEDRUNS
WHERE VERIFIED = "Yes"
ORDER BY ID DESC
LIMIT 5;

#### **Q2: Find players who appear in more than one category.**
*Also included a secondary question within this that asks to grab the total number of players that have runs in 5, 4, 3, etc. categories.*

In [ ]:
%%sql

/*Learned about the GROUP_CONCAT() function which lets you concat grouped values.
  In this, Im using it to display all of the categories that a user has entered a run in.*/

SELECT GROUP_CONCAT(CATEGORY) AS CATS_RAN_BY_PLAYER, PLAYER_NAME, COUNT(PLAYER_NAME) AS NUM_OF_CATEGORIES
FROM ALL_CAT_SPEEDRUNS
GROUP BY PLAYER_NAME
HAVING COUNT(PLAYER_NAME) BETWEEN 1 AND 5
ORDER BY NUM_OF_CATEGORIES DESC;

In [ ]:
%%sql

/*
    Going the extra mile here for more fun
    The number of players that are in 1 or more categories:
        5: 88
        4: 91
        3: 145
        2: 254
        1: 353
*/

SELECT CAT_COUNT, COUNT(PLAYER_NAME) AS NUM_OF_PLAYERS
FROM 
(
    SELECT CATEGORY, PLAYER_NAME, COUNT(CATEGORY) AS CAT_COUNT
    FROM ALL_CAT_SPEEDRUNS
    GROUP BY PLAYER_NAME
    HAVING COUNT(PLAYER_NAME) <= 5 AND COUNT(PLAYER_NAME) > 0
)
GROUP BY CAT_COUNT
ORDER BY CAT_COUNT DESC;


#### **Q3: Find the top 3 players in each category from each country.**
*One answer by transposing and the other by using window functions (i.e. RANK())*

*Note: some times may not line up as they are on speedrun.com as the dataset is a bit dated.*

In [27]:
pd.set_option("display.max_rows", None)


In [28]:
%%sql

/*There are other countries other than five listed below, but the players in those countries don't have a 
first, second or third place time.*/

SELECT PLATFORM, CATEGORY, COUNTRY,
    GROUP_CONCAT(CASE WHEN PLACE = 1 THEN PLAYER_NAME END) as FIRST,
    GROUP_CONCAT(CASE WHEN PLACE = 2 THEN PLAYER_NAME END) as SECOND,
    GROUP_CONCAT(CASE WHEN PLACE = 3 THEN PLAYER_NAME END) as THIRD
FROM ALL_CAT_SPEEDRUNS
GROUP BY PLATFORM, CATEGORY, COUNTRY
HAVING PLACE IN ('1', '2', '3')
ORDER BY COUNTRY ASC;

 * sqlite:///SPEEDRUNS.db
Done.


PLATFORM,CATEGORY,COUNTRY,FIRST,SECOND,THIRD
EMU,120 Star,Australia,None,Nebula,None
VC,0 Star,Australia,None,None,PhoenixAUS
N64,0 Star,Canada,Suigi,None,None
N64,1 Star,Canada,Suigi,None,None
N64,16 Star,Canada,Suigi,None,None
N64,70 Star,Canada,Suigi,None,None
EMU,1 Star,Germany,Raisn,None,None
EMU,120 Star,Germany,Raisn,None,None
EMU,70 Star,Germany,Raisn,None,None
VC,0 Star,Germany,None,Finnii602,None


In [9]:
%%sql

/*Finding how many times a player is 1st, 2nd or 3rd in a category.*/

SELECT PLAYER_NAME, COUNTRY, COUNT(FIRST) AS NUM_OF_1ST_PLACE, COUNT(SECOND) AS NUM_OF_2ND_PLACE, COUNT(THIRD) AS NUM_OF_3RD_PLACE
FROM 
(
    SELECT PLAYER_NAME, CATEGORY, COUNTRY,
        GROUP_CONCAT(CASE WHEN PLACE == 1 THEN PLAYER_NAME END) as FIRST,
        GROUP_CONCAT(CASE WHEN PLACE == 2 THEN PLAYER_NAME END) as SECOND,
        GROUP_CONCAT(CASE WHEN PLACE == 3 THEN PLAYER_NAME END) as THIRD
    FROM ALL_CAT_SPEEDRUNS
    GROUP BY CATEGORY, COUNTRY
    HAVING PLACE IN (1, 2, 3) AND COUNTRY IS NOT NULL 
    ORDER BY COUNTRY ASC
) as top3Table
GROUP BY PLAYER_NAME
ORDER BY NUM_OF_1ST_PLACE desc;

 * sqlite:///SPEEDRUNS.db
Done.


PLAYER_NAME,COUNTRY,NUM_OF_1ST_PLACE,NUM_OF_2ND_PLACE,NUM_OF_3RD_PLACE
Raisn,Germany,3,2,1
Suigi,Canada,1,0,0
Karin,Japan,1,0,0
KANNO,nan,1,1,0
CyanogenSm64,Uzbekistan,1,0,1
taihou,Japan,0,1,1
cjrokokomero,Italy,0,1,1
Zhilkan,Spain,0,1,0
ZUMMI,Italy,0,0,1
Sigotu,Poland,0,0,1


#### **Q4: Retrieve the first place times per category for each platform (N64, EMU, VC)**

In [ ]:
%%sql

/*For some reason this players time was changed to a lower value. I spot-checked a bunch before and after their time in the dataset and
everyone elses time seems fine. Not sure what went wrong there.*/

UPDATE ALL_CAT_SPEEDRUNS
    SET RUN_TIME = "1:36:48"
    WHERE ID = "1770";

In [ ]:
%%sql

/*This is essentially it. This is something thatd be better viewed in a chart or graph.*/

SELECT CATEGORY,
    MIN(CASE WHEN PLATFORM = "EMU" THEN RUN_TIME END) AS EMU,
    MIN(CASE WHEN PLATFORM = "N64" THEN RUN_TIME END) AS N64,
    MIN(CASE WHEN PLATFORM = "VC" THEN RUN_TIME END) AS VC
FROM
(
    SELECT *
    FROM ALL_CAT_SPEEDRUNS
    WHERE VERIFIED = "Yes" AND SUBMISSION_DATE IS NOT NULL AND PLAYER_NAME IS NOT NULL AND COUNTRY IS NOT NULL
)
GROUP BY CATEGORY

#### **Q5: Who has held the N64 World Record in each category the longest?**

In [ ]:
%%sql

/*
     0: Suigi
     1: Suigi
     16: Suigi
     70: Suigi
     120: Karin
*/

SELECT 
    PLATFORM,
    PLAYER_NAME, 
    CATEGORY, 
    PLACE, 
    RUN_TIME, 
    SUBMISSION_DATE, 
    --Casting the rounded value of the difference between the current date and the submission date to an integer to get a whole number.
    CAST(ROUND(julianday('now') - julianday(SUBMISSION_DATE)) AS INTEGER) AS DAYS_RECORD_HELD
FROM ALL_CAT_SPEEDRUNS
GROUP BY CATEGORY, PLATFORM
HAVING PLACE == 1 AND PLATFORM == "N64"
ORDER BY CATEGORY DESC;

## **Visualizing Data Using *Matplotlib* and *Seaborn* (extra)**

In [ ]:
query = pd.read_sql(
    '''
    SELECT
        PLATFORM,
        PLAYER_NAME,
        CATEGORY,
        PLACE,
        RUN_TIME,
        SUBMISSION_DATE,
        CAST(ROUND(julianday() - julianday(SUBMISSION_DATE)) AS INTEGER) AS DAYS_RECORD_HELD
    FROM ALL_CAT_SPEEDRUNS
    GROUP BY PLATFORM, CATEGORY
    HAVING MIN(PLACE) == 1 AND PLATFORM = "N64"
    ORDER BY CATEGORY DESC;
    ''',
    connObj
)

query_2 = pd.read_sql(
    '''
    SELECT CATEGORY,
        MIN(CASE WHEN PLATFORM = "EMU" THEN RUN_TIME END) AS EMU,
        MIN(CASE WHEN PLATFORM = "N64" THEN RUN_TIME END) AS N64,
        MIN(CASE WHEN PLATFORM = "VC" THEN RUN_TIME END) AS VC
    FROM
    (
        SELECT *
        FROM ALL_CAT_SPEEDRUNS
        WHERE VERIFIED = "Yes" AND SUBMISSION_DATE IS NOT NULL AND PLAYER_NAME IS NOT NULL AND COUNTRY IS NOT NULL
    )
    GROUP BY CATEGORY
    ''',
    connObj
)

df_viz = pd.DataFrame(query)
df_viz_2 = pd.DataFrame(query_2)

In [ ]:
df_viz

In [ ]:
df_viz_2

In [ ]:
x1 = df_viz[df_viz['PLATFORM'] == "N64"]['CATEGORY']
y1 = df_viz[df_viz['PLATFORM'] == "N64"]['RUN_TIME']
x2 = df_viz[df_viz['PLATFORM'] == "EMU"]['CATEGORY']
y2 = df_viz[df_viz['PLATFORM'] == "EMU"]['RUN_TIME']
x3 = df_viz[df_viz['PLATFORM'] == "VC"]['CATEGORY']
y3 = df_viz[df_viz['PLATFORM'] == "VC"]['RUN_TIME']

# Plotting
plt.title("First Places Times Per Category (Each Platform)")
plt.xlabel("Category")
plt.ylabel("Run Time")
plt.legend()

plt.show()

In [ ]:
x = df_viz['CATEGORY']
y = df_viz['DAYS_RECORD_HELD']

plt.plot(x, y)
plt.title("How long the WR has been held in each category")
plt.xlabel("Category")
plt.ylabel("# of Days the WR Has Been Held")
plt.show()

In [ ]:
#connObj.close